In [1]:
from datetime import datetime
import json
import logging
from pathlib import Path
import re
import time
from typing import List, Tuple
from tempfile import NamedTemporaryFile

import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, as_completed
import dask_geopandas as dgd 
import hydra
import geopandas as gpd
import numpy as np
from omegaconf import DictConfig, OmegaConf
import pandas as pd
from pyproj import CRS
from tqdm.notebook import tqdm
import xarray as xr
import zarr

In [2]:
def split_gdf_into_pieces(gdf, n_pieces):
    """
    Splits a GeoDataFrame into n nearly equal pieces.

    Parameters
    ----------
    gdf : GeoDataFrame
        The GeoDataFrame to split.
    n_pieces : int
        The number of pieces to split the gdf into.

    Returns
    -------
    list of GeoDataFrame
        A list containing the split GeoDataFrames.
    """
    # Calculate the size of each split
    split_size = len(gdf) // n_pieces  # Use floor division to get an integer result
    
    # Split the GeoDataFrame into n_pieces
    gdf_splits = [gdf.iloc[i:i + split_size] for i in range(0, len(gdf), split_size)]
    
    # To handle any remainder, ensure all pieces are captured by adjusting the last piece
    if len(gdf_splits) > n_pieces:
        # Append the remainder to the last piece if more splits than n_pieces
        gdf_splits[n_pieces-1] = pd.concat([gdf_splits[n_pieces-1], *gdf_splits[n_pieces:]])
        gdf_splits = gdf_splits[:n_pieces]
    
    return gdf_splits

In [9]:
zones = [71, 72, 73, 74, 75, 76, 77, 78]
for zone in tqdm(zones, desc="writing to zones"):
    river_path = Path("/data/tkb5476/projects/marquette/data/HUC/buffered_flowlines/") / f"riv_pfaf_{zone}_ME_PairwiseBuffe2.shp"
    gdf = gpd.read_file(river_path).to_crs(epsg=4326)
    gdf_pieces = split_gdf_into_pieces(gdf, 50)
    
    output_dir = Path("/data/tkb5476/projects/marquette/data/HUC/split_buffered_flowlines/") / f"riv_pfaf_{zone}_buff_split"
    output_dir.mkdir(parents=True, exist_ok=True)  # Ensure the output directory exists
    
    for idx, gdf_piece in enumerate(gdf_pieces, start=1):
        output_path = output_dir / f"split_{idx}.shp"
        gdf_piece.to_file(output_path)
    
    print(f"Finished splitting and saving zone {zone} shapefiles into {len(gdf_pieces)} pieces.")

writing to zones:   0%|          | 0/8 [00:00<?, ?it/s]

Finished splitting and saving zone 71 shapefiles into 50 pieces.
Finished splitting and saving zone 72 shapefiles into 50 pieces.
Finished splitting and saving zone 73 shapefiles into 50 pieces.
Finished splitting and saving zone 74 shapefiles into 50 pieces.
Finished splitting and saving zone 75 shapefiles into 50 pieces.
Finished splitting and saving zone 76 shapefiles into 50 pieces.
Finished splitting and saving zone 77 shapefiles into 50 pieces.
Finished splitting and saving zone 78 shapefiles into 50 pieces.


In [10]:
# Testing:
split_path = Path("/data/tkb5476/projects/marquette/data/HUC/split_buffered_flowlines/riv_pfaf_71_buff_split/")
gpd.read_file(split_path / "split_1.shp").to_crs(epsg=4326).tail()

,COMID,lengthkm,lengthdir,sinuosity,slope,uparea,order_,strmDrop_t,slope_taud,NextDownID,maxup,up1,up2,up3,up4,BUFF_DIST,ORIG_FID,Shape_Leng,Shape_Area,geometry
974,71000975.0,5.006591,3.691281,1.356329,0.002112,2675.062820,4.0,10.6,0.002112,71000816.0,2.0,71000993.0,71001051.0,0.0,0.0,250.0,974,0.135084,0.000390,"POLYGON ((-94.25656 58.40000, -94.25656 58.399..."
975,71000976.0,4.846783,4.034712,1.201271,0.000000,2609.714063,4.0,0.0,0.000000,71000973.0,2.0,71000979.0,71001675.0,0.0,0.0,250.0,975,0.126992,0.000394,"POLYGON ((-97.03075 58.20583, -97.03075 58.205..."
976,71000977.0,6.317802,5.595961,1.128993,0.000000,757.240928,3.0,0.0,0.000000,71000845.0,2.0,71000986.0,71001383.0,0.0,0.0,250.0,976,0.156859,0.000502,"POLYGON ((-103.11081 58.25956, -103.11075 58.2..."
977,71000978.0,7.785729,6.084167,1.279670,0.000128,2678.545169,4.0,0.0,0.000000,71000972.0,2.0,71000992.0,71001153.0,0.0,0.0,250.0,977,0.234271,0.000604,"POLYGON ((-100.74865 58.89063, -100.74948 58.8..."
978,71000979.0,1.752009,1.521771,1.151296,0.000571,2557.402143,4.0,0.0,0.000000,71000976.0,2.0,71000983.0,71001137.0,0.0,0.0,250.0,978,0.071838,0.000159,"POLYGON ((-97.03708 58.18812, -97.03791 58.187..."


In [11]:
gpd.read_file(split_path / "split_2.shp").to_crs(epsg=4326).head()

,COMID,lengthkm,lengthdir,sinuosity,slope,uparea,order_,strmDrop_t,slope_taud,NextDownID,maxup,up1,up2,up3,up4,BUFF_DIST,ORIG_FID,Shape_Leng,Shape_Area,geometry
0,71000980.0,26.129100,13.845524,1.887188,0.001901,1605.563711,3.0,49.8,0.001901,71000964.0,2.0,71000982.0,71001765.0,0.0,0.0,250.0,979,0.580029,0.001774,"POLYGON ((-96.05574 58.31680, -96.05574 58.316..."
1,71000981.0,15.188226,10.337730,1.469203,0.001012,800.859114,3.0,15.4,0.001012,71000963.0,2.0,71000989.0,71001527.0,0.0,0.0,250.0,980,0.339159,0.001069,"POLYGON ((-95.66074 58.25417, -95.66074 58.253..."
2,71000982.0,4.521035,3.291034,1.373743,0.000221,1519.160901,3.0,0.0,0.000000,71000980.0,2.0,71000988.0,71001759.0,0.0,0.0,250.0,981,0.126190,0.000342,"POLYGON ((-96.29824 58.25583, -96.29824 58.255..."
3,71000983.0,8.590328,6.845082,1.254963,0.000233,1291.272589,3.0,0.0,0.000000,71000979.0,2.0,71000985.0,71001635.0,0.0,0.0,250.0,982,0.212108,0.000665,"POLYGON ((-97.03409 58.14750, -97.03409 58.146..."
4,71000984.0,12.722484,7.085750,1.795503,0.000565,895.836553,4.0,7.2,0.000565,71000974.0,2.0,71000990.0,71001353.0,0.0,0.0,250.0,983,0.305684,0.000873,"POLYGON ((-93.52411 58.06513, -93.52411 58.064..."
